In [1]:
from ollama_client import OllamaClient
from pydantic import BaseModel, Field
from tools import tavily_search
import datetime
import json
from openai_client import OpenAIClient as OpenAIClient
from openrouter_client import Client
from deep_research_tools import deep_tavily_search
from IPython.display import display, Markdown

print(datetime.datetime.now().strftime("%d %b %Y"))

# loadenv
import dotenv
dotenv.load_dotenv()

provider = 'openai'
provider = "openrouter"
provider = 'ollama'

if provider == "openai":
    Client  = OpenAIClient
    model1 = 'gpt-5-nano'
    model2 = 'gpt-5-mini'


elif provider == "ollama":
    Client = OllamaClient
    model1 = "qwen3:4b"
    model2 = "qwen2.5:14b"
    model3 = "mistral-small3.2:latest"
    model4 = "magistral:latest"
    model5 = "qwen3:8b"


elif provider == "openrouter":
    Client = Client
    model1 = "openai/gpt-oss-20b:free"
    model2 = "openai/gpt-oss-120b:free"
    model2 = "qwen/qwen3-30b-a3b:free"
    model4 = "qwen/qwen3-235b-a22b:free"
    model5 = "meta-llama/llama-4-maverick:free"
    model6 = "deepseek/deepseek-chat-v3.1:free"
    model7 = "mistralai/mistral-small-3.2-24b-instruct:free"
    model8 = "qwen/qwq-32b:free"
    model9 = "deepseek/deepseek-r1-0528:free"

03 Sep 2025


### Reformulate User Question

In [3]:
task = """you are an expert on simplifying user question for optimal google search. 
1. Identify 3 different  formulations of what the user is asking
2. keep the reformulations short and concise
3. the 3 formulations must be varied and must be useful and varied variations of the user query but must be strongly related to the user query
4. No need to write complete sentences but include all important aspects"""

re_phrase_agent = Client(role = task, model_name=model1)

class RePhraseResult(BaseModel):
    formulation_1: str
    formulation_2: str
    formulation_3: str


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

{'model': 'qwen3:4b', 'created_at': '2025-09-02T19:53:29.670896902Z', 'message': {'role': 'assistant', 'content': '{\n  "formulation_1": "improve python code performance",\n  "formulation_2": "python performance optimization techniques",\n  "formulation_3": "how to speed up python code execution"\n}\n\n'}, 'done_reason': 'stop', 'done': True, 'total_duration': 2765078610, 'load_duration': 1651671248, 'prompt_eval_count': 447, 'prompt_eval_duration': 116335651, 'eval_count': 47, 'eval_duration': 986138967}


{'text': '{\n  "formulation_1": "improve python code performance",\n  "formulation_2": "python performance optimization techniques",\n  "formulation_3": "how to speed up python code execution"\n}\n\n',
 'raw': {'model': 'qwen3:4b',
  'created_at': '2025-09-02T19:53:29.670896902Z',
  'message': {'role': 'assistant',
   'content': '{\n  "formulation_1": "improve python code performance",\n  "formulation_2": "python performance optimization techniques",\n  "formulation_3": "how to speed up python code execution"\n}\n\n'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 2765078610,
  'load_duration': 1651671248,
  'prompt_eval_count': 447,
  'prompt_eval_duration': 116335651,
  'eval_count': 47,
  'eval_duration': 986138967},
 'conversation_history': [{'role': 'user',
   'content': 'Hey Man. i love coding. \n                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?'},
  {'role': 'assistant',
   'content':

In [8]:
print(re_phrase_agent.system_instructions)

You are a helpful Agent among a group of agents trying to solve a problem. Each agent is tasked with a part or the entire problem.
You will be given your task. You will have access to all the relevant information and tools. You can also see the work already done by other agents. Use that information if required.


## Instructions
1. View the context, the task executed, the results, the tool call results of the other agents.
2. Reason and execute your task.
3. You have access to multiple tools as listed. You can only call tools which are relevant to your task. Other agents might have executed other tools which you dont have access to.
4. Make the necessary tool calls if your tool descriptions match the task
5. your task will be enclosed in <YOUR TASK></YOUR TASK> tags. This is your task. Only execute this task.
6. The work done by other agents will be enclosed in <Agent: Agent Name></Agent: Agent Name> tags. There may be multiple of these.
7. Any information to be used as reference (fro

### Web Search Agent

In [7]:
task = """You are an expert on simplifying user question for optimal google search and then invoking search tool. 
1. Identify 3 different formulations of what the user is asking
2. keep the reformulations short and concise. 
3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the user question 3 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 3 ways and pass each of them to the tool
"""

re_phrase_agent = Client(role = task, model_name=model1, agent_name="SearchAgent")


search_queries = re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: What is the latest version of gpt out there?""", tools=[tavily_search])

search_queries

HTTPError: 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions

In [9]:
import json
print(search_queries['tool_results']["tavily_search"])
print(len(search_queries['tool_results']["tavily_search"]))

KeyError: 'tavily_search'

## Deep Research 1


#### Deep Research Clarification Asker

In [5]:
task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
"""

clarifying_agent = Client(role = task, model_name=model2, agent_name="ClarifyingAgent")

class ClarifyingQuestions(BaseModel):
    question_1: str
    question_2: str
    question_3: str


#clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
clarifying_questions = clarifying_agent.invoke(query="""Help me understand where to buy good coffee beans""", 
                                               json_schema=ClarifyingQuestions)



clarifying_questions

{'text': '{\n\n  "question_1": "What is your preferred coffee brewing method (e.g., espresso, drip, French press, pour-over)?",\n  "question_2": "Do you have a budget range for purchasing coffee beans?",\n  "question_3": "Are you looking for specific coffee origins or roast profiles (e.g., Ethiopian, Sumatran, light roast, dark roast)?"\n  \n}',
 'raw': {'id': 'gen-1756494974-Ma7AzsXUzrK08xiJSNFV',
  'provider': 'Chutes',
  'model': 'qwen/qwen3-30b-a3b:free',
  'object': 'chat.completion',
  'created': 1756494975,
  'choices': [{'logprobs': None,
    'finish_reason': 'stop',
    'native_finish_reason': 'stop',
    'index': 0,
    'message': {'role': 'assistant',
     'content': '{\n\n  "question_1": "What is your preferred coffee brewing method (e.g., espresso, drip, French press, pour-over)?",\n  "question_2": "Do you have a budget range for purchasing coffee beans?",\n  "question_3": "Are you looking for specific coffee origins or roast profiles (e.g., Ethiopian, Sumatran, light roas

In [4]:
def deep_research_clarification_agent(deep_research_request, model, client):
    
    task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / understand / plan out.
    4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

    For example, if the user asks: 
    Example 1:
    User Question: Help me prepare an itinerary for my trip to Japan next month.
    Clarifying questions: 
    1. What specific cities in Japan are you planning to visit?
    2. How many days do you have for your trip?
    3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
    4. What is your budget for the trip?

    Example 2:
    User Question: Which city are you moving to?
    Clarifying Questions:
    1. Which city are you moving to?
    2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
    3. What’s your estimated move-in date?
    4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
    5. What is your budget for the move and for monthly living expenses?


    Example 3:
    User Question:
    I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

    Clarifying Questions:
    1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
    2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
    3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
    4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
    """

    clarifying_agent = client(role = task, model_name=model, agent_name="ClarifyingAgent")

    class ClarifyingQuestions(BaseModel):
        question_1: str
        question_2: str
        question_3: str


    #clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
    clarifying_questions = clarifying_agent.invoke(query=deep_research_request, 
                                                json_schema=ClarifyingQuestions)

    pre_r = json.loads(clarifying_questions['text'])
    r = "To help me answer your query, pleae answer the following questions:\n"

    for i in range(len(pre_r)):
        r += f"{i+1}. {pre_r[f'question_{i+1}']}\n"

    # Context till this node to be passed to other agents
    deep_research_agent_context = clarifying_agent.get_agent_context()
    return deep_research_agent_context, r

In [6]:
research_question  =  "Show me how to buy good coffee online"
deep_research_agent_context, r = deep_research_clarification_agent(research_question, model=model1, client=Client)
print(deep_research_agent_context, r)

{'model': 'qwen3:4b', 'created_at': '2025-09-02T19:54:45.08505425Z', 'message': {'role': 'assistant', 'content': '{\n  "question_1": "What is your primary goal for buying coffee online? (e.g., convenience, quality, cost, specific brand preferences)",\n  "question_2": "How many cups of coffee do you typically consume per day?",\n  "question_3": "Do you have any specific preferences for coffee type (e.g., dark roast, light roast, espresso, pour-over, cold brew) or brewing method?"\n}\n\n'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3647374681, 'load_duration': 1664211989, 'prompt_eval_count': 826, 'prompt_eval_duration': 293497484, 'eval_count': 91, 'eval_duration': 1683077628}
<Agent: ClarifyingAgent>
            ### Task to be done by the agent:
            You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user 

In [7]:
def add_agent_response_and_user_answer(agent_response: str, user_answers: str) -> str:
    agent_response = """\n\nagent_response: """ + agent_response
    user_answers = """\n\nuser_answers: """ + user_answers

    return agent_response + user_answers


answer = """
        1. quality
        2. 2
        3. light roast pourover
        """


next_context_to_be_passed = deep_research_agent_context + add_agent_response_and_user_answer(r, answer)

print(next_context_to_be_passed)

<Agent: ClarifyingAgent>
            ### Task to be done by the agent:
            You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / understand / plan out.
    4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

    For example, if the user asks: 
    Example 1:
    User Question: Help me prepare an itinerary for my trip to Japan next month.
    Clarifying questions: 
    1. What specific cities in Japan are you planning to visit?
    2. How many days do you have for your trip?
    3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
    4. What is your budget for the trip?

    Example 

In [8]:
Markdown(clarifying_agent.get_agent_context())

NameError: name 'clarifying_agent' is not defined

### Deep Research Search Agent

In [10]:
task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
Observe the shared conversation between user and Agent and generate 6 questions which, when searched can answer the user's intent.
1. Identify 6 questions based on the conversation on which web search can be done.
2. keep the search questions short and concise. 
3. Invoke the deep_tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. One of the 6 questions you make, must be about an expanded search space. For example, if the user wants to learn about Linear Regression, also include a search item for something like Lasso or Ridge which are related but still introduce some variation
8. The 6th question, as mentioned must go a bit beyond the exact search - something that might introduce a new perspective
"""

deep_search_question_formulator_1 = Client(role = task, model_name=model1, agent_name="DeepSearchAgent1", history_from_other_agents = next_context_to_be_passed)
#deep_search_question_formulator_1.set_history_from_other_agents(history = next_context_to_be_passed)
class DeepSearchAgent1(BaseModel):
    question1: str
    question2: str
    question3: str
    question4: str
    question5: str
    question6: str

search_queries = deep_search_question_formulator_1.invoke(query=f"""Search the web with the rephrased 6 questions.""",
                                                           tools=[deep_tavily_search])
search_queries_full_context = deep_search_question_formulator_1.get_agent_context()
search_queries

{'model': 'qwen3:4b', 'created_at': '2025-09-02T19:58:02.959788446Z', 'message': {'role': 'assistant', 'content': '<think>\nOkay, let\'s tackle this. So the user wants to buy good coffee online, and they provided their answers to the clarifying questions from the ClarifyingAgent. The user said their primary goal is quality, they drink 2 cups a day, and they prefer light roast pour-over.\n\nFirst, I need to generate 6 search questions based on their answers. The task says to rephrase the user\'s question in 6 different ways, each concise, and one of them should be an expanded search space, and the sixth should go beyond the exact search for a new perspective.\n\nLet me start by understanding what the user needs. They want to buy coffee online with quality as the main goal, 2 cups a day, light roast pour-over. So the main topic is finding online coffee shops or services that offer high-quality light roast pour-over coffee.\n\nFirst, the 6 questions. Let me think:\n\n1. The basic question

{'text': '<think>\nOkay, let\'s tackle this. So the user wants to buy good coffee online, and they provided their answers to the clarifying questions from the ClarifyingAgent. The user said their primary goal is quality, they drink 2 cups a day, and they prefer light roast pour-over.\n\nFirst, I need to generate 6 search questions based on their answers. The task says to rephrase the user\'s question in 6 different ways, each concise, and one of them should be an expanded search space, and the sixth should go beyond the exact search for a new perspective.\n\nLet me start by understanding what the user needs. They want to buy coffee online with quality as the main goal, 2 cups a day, light roast pour-over. So the main topic is finding online coffee shops or services that offer high-quality light roast pour-over coffee.\n\nFirst, the 6 questions. Let me think:\n\n1. The basic question: "Best online coffee shops for light roast pour-over coffee with high quality for 2 cups per day"\n2. Ma

In [11]:
def deep_search_search_agent_1(model=model1):
    task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
    Observe the shared conversation between user and Agent and generate 6 questions which, when searched can answer the user's intent.
    1. Identify 6 questions based on the conversation on which web search can be done.
    2. keep the search questions short and concise. 
    3. Invoke the deep_tavily_search if it will help provide an updated and more accurate answer to the question.
    4. If invoking the tool, extract from the agent-user question 6 different aspects of the question and pass to the search tool.
    5. Each question should focus on a different aspect of the question.
    6. If the question needs to refer to documentation, do a web search and answer
    7. If the user asks for recent events or asks to refer to the web, then use this tool
    8. The 6th question, as mentioned must go a bit beyond the exact search - something that might introduce a new perspective
    9. Rephrase the user question in 6 ways and pass each of them to the tool
    """

    deep_search_question_formulator_1 = Client(role = task, model_name=model, agent_name="DeepSearchAgent1", history_from_other_agents = next_context_to_be_passed)
    #deep_search_question_formulator_1.set_history_from_other_agents(history = next_context_to_be_passed)
    class DeepSearchAgent1(BaseModel):
        question1: str
        question2: str
        question3: str
        question4: str
        question5: str
        question6: str

    search_queries = deep_search_question_formulator_1.invoke(query=f"""Search the web with the rephrased 6 questions.""",
                                                            tools=[deep_tavily_search])
    search_queries_full_context = deep_search_question_formulator_1.get_agent_context()
    return search_queries_full_context, search_queries

In [8]:
search_queries_full_context, search_queries = deep_search_search_agent_1(model=model1)

In [9]:
search_queries['tool_results']

{'deep_tavily_search': "<search_metadata>\nQueries: ['What are the best light roast whole bean coffees available online that ship to Chennai for pour-over and can be purchased as a one-time order?', 'Where can I buy high-quality light roast coffee beans online for pour-over, with delivery to Chennai, as a single order?', 'Which online retailers offer fresh light roast whole bean coffee suitable for pour-over and ship to Chennai without a subscription?', 'What are top-rated light roast whole bean coffees for pour-over that ship to Chennai for one-time purchases?', 'How to choose a reliable online coffee seller for light roast beans, whole bean, for pour-over, shipping to Chennai, with no subscription?', 'Expanded search: Compare light roast vs other roasts for pour-over, focusing on origin, grind size, freshness, price, and Chennai shipping options from online shops (subscription vs one-time).']\nTotal Queries: 6\nMax Results per Query: 15\nSearch Mode: Advanced\nRaw Content: False\n</s

### Final Research Report Creator

In [19]:
task = """You are a Research expert who produces expert level research reports by observign questions and context. 
The research report must answer the user question enclosed in <research_question> tags. The search results are enclosed in <search_results> tags.
Follow the following guidelines while preparing the report:
1. First, try really hard to understand the intent of the research question, and look at the web search results. 
2. Make a plan on how to organize the information to properly answer the user question
3. The report must be in markdown format with proper headings, tables, etc.
4. The report must start with a 3 - 10 line direct answer to the user question with a heading: Summary.
5. Proceed to then present the detailed research on the user topic. Use information present only in the search results. 
6. Summarize the overall thoughts in a table at the end of the report in markdown format.

<research_question>
{research_question}
</research_question>


<search_results>
{search_results}
</search_results>

""".format(research_question  =  research_question, search_results = search_queries['tool_results']['deep_tavily_search'])

deep_search_report_generator = Client(role = task, model_name=model1, agent_name="DeepResearchReportAgent1")


deep_search_report_generator_results = deep_search_report_generator.invoke(query="""Prepare a deep research report based on the original question: {research_question} and the search results: {search_results}""")
deep_search_report_generator_full_context = deep_search_report_generator.get_agent_context()


{'model': 'qwen3:4b', 'created_at': '2025-09-02T20:07:37.531470795Z', 'message': {'role': 'assistant', 'content': '<think>\nOkay, the user wants me to prepare a deep research report based on a specific research question and some search results. But wait - they haven\'t actually provided the research question or the search results. That\'s a problem. \n\nHmm, looking at the query structure, they\'ve written "{research_question}" and "{search_results}" as placeholders. That means they probably intended to paste real content but forgot. Smart users would have included the actual text, but since they didn\'t... \n\nI should handle this carefully. If I pretend to know what they meant, that could be misleading. The safest approach is to be transparent about the missing information while still showing how I\'d proceed if they provided it. \n\nThe user seems knowledgeable - they used precise terms like "deep research report" and "search results". Probably a researcher, student, or professional

In [21]:
print(task, len(task))

You are a Research expert who produces expert level research reports by observign questions and context. 
The research report must answer the user question enclosed in <research_question> tags. The search results are enclosed in <search_results> tags.
Follow the following guidelines while preparing the report:
1. First, try really hard to understand the intent of the research question, and look at the web search results. 
2. Make a plan on how to organize the information to properly answer the user question
3. The report must be in markdown format with proper headings, tables, etc.
4. The report must start with a 3 - 10 line direct answer to the user question with a heading: Summary.
5. Proceed to then present the detailed research on the user topic. Use information present only in the search results. 
6. Summarize the overall thoughts in a table at the end of the report in markdown format.

<research_question>
Show me how to buy good coffee online
</research_question>


<search_result

In [20]:
print(deep_search_report_generator_results['text'])

<think>
Okay, the user wants me to prepare a deep research report based on a specific research question and some search results. But wait - they haven't actually provided the research question or the search results. That's a problem. 

Hmm, looking at the query structure, they've written "{research_question}" and "{search_results}" as placeholders. That means they probably intended to paste real content but forgot. Smart users would have included the actual text, but since they didn't... 

I should handle this carefully. If I pretend to know what they meant, that could be misleading. The safest approach is to be transparent about the missing information while still showing how I'd proceed if they provided it. 

The user seems knowledgeable - they used precise terms like "deep research report" and "search results". Probably a researcher, student, or professional needing this for work. They might be in a hurry since they're using placeholder brackets. 

I recall from previous interaction

In [1]:
len(task)

NameError: name 'task' is not defined

In [16]:
Markdown(deep_search_report_generator_results['text'])

In [13]:
deep_search_report_generator_results

{'text': '',
 'raw': {'id': 'chatcmpl-C9R19T1bQxOHS7BhYUuXZrKRccYa4',
  'object': 'chat.completion',
  'created': 1756364843,
  'model': 'gpt-5-nano-2025-08-07',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '',
     'refusal': None,
     'annotations': []},
    'finish_reason': 'length'}],
  'usage': {'prompt_tokens': 36045,
   'completion_tokens': 4000,
   'total_tokens': 40045,
   'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
   'completion_tokens_details': {'reasoning_tokens': 4000,
    'audio_tokens': 0,
    'accepted_prediction_tokens': 0,
    'rejected_prediction_tokens': 0}},
  'service_tier': 'default',
  'system_fingerprint': None},
 'conversation_history': [{'role': 'user',
   'content': "You are a Research expert who produces expert level research reports by observign questions and context. \nThe research report must answer the user question enclosed in <research_question> tags. The search results are enclosed in <search

### Generic Tool calling function

In [ ]:
def call_tools():
    

### Agent that summarizes the Answer

In [6]:
task = """You are an Expert on Answering a User Question based on provided context. 
Study the context deeply and answer the question asked by the user. Use a table in the end to summarize the answer"""
model1="qwen3:1.7b"
model2="llama3.2-vision:11b"
model3="gemma3:4b"


re_phrase_agent = OllamaClient(role = task, model_name=model2)

class AnswerBasedOnContext(BaseModel):
    answer: str
    


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/chat